In [1]:
# # IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# # RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
# import kagglehub
# kagglehub.login()


In [2]:
# # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# # THEN FEEL FREE TO DELETE THIS CELL.
# # NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# # ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# # NOTEBOOK.

# the_ancient_texts_provenance_challenge_path = kagglehub.competition_download('the-ancient-texts-provenance-challenge')

# print('Data source import complete.')


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk("/kaggle/input/" ):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-ancient-texts-provenance-challenge/sample_submission.csv
/kaggle/input/the-ancient-texts-provenance-challenge/train.csv
/kaggle/input/the-ancient-texts-provenance-challenge/test.csv


In [4]:
pip install -U transformers huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 80.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [5]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score, classification_report
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments


2025-10-19 17:08:05.288198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760893685.531767      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760893685.598502      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
train_df = pd.read_csv("/kaggle/input/the-ancient-texts-provenance-challenge/train.csv")
test_df = pd.read_csv('/kaggle/input/the-ancient-texts-provenance-challenge/test.csv')

In [7]:
# Separate rare classes
rare = train_df.groupby("label").filter(lambda x: len(x) == 1)
rest = train_df.drop(rare.index)

# Split the rest
train_rest, eval_df = train_test_split(rest, test_size=0.3,stratify=rest["label"],random_state=42)  # stratify only the non-rare classes

# Combine rare classes back into training
train_df = pd.concat([train_rest, rare])

In [8]:
train_df.columns

Index(['id', 'text', 'label'], dtype='object')

In [9]:
train_df["label"].value_counts(),eval_df["label"].value_counts()

(label
 7     24698
 8     13231
 5     11810
 0      6373
 6      5108
 3      4159
 1      4087
 10     3950
 4      3629
 2      3021
 11     1777
 9      1698
 12      118
 13       99
 14        1
 Name: count, dtype: int64,
 label
 7     10585
 8      5670
 5      5061
 0      2731
 6      2189
 3      1783
 1      1752
 10     1693
 4      1555
 2      1294
 11      761
 9       728
 12       51
 13       43
 14        1
 Name: count, dtype: int64)

## Tokenizing Text

In [10]:
from datasets import Dataset

In [11]:
# converting dataset comaptible to hugging face
train_ds = Dataset.from_pandas(train_df)
eval_ds = Dataset.from_pandas(eval_df)

In [12]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

train_ds = train_ds.map(tokenize, batched=True)
eval_ds  = eval_ds.map(tokenize, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/83759 [00:00<?, ? examples/s]

Map:   0%|          | 0/35897 [00:00<?, ? examples/s]

### Using Focal Loss for Class Imbalance handling

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):

        targets = targets.to(inputs.device)
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        if self.alpha is not None:
            alpha = self.alpha.to(inputs.device)[targets]
            ce_loss = alpha * ce_loss
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Compute class weights
class_counts = train_df["label"].value_counts().sort_index()
weights = 1.0 / class_counts
weights = torch.tensor(weights.values, dtype=torch.float)
loss_fn = FocalLoss(alpha=weights, gamma=2.0)

In [15]:
num_labels = train_df["label"].nunique()
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)
model.to(device);


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,**kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    return {
        "macro_f1": f1_score(p.label_ids, preds, average="macro"),
        "weighted_f1": f1_score(p.label_ids, preds, average="weighted")
    }

## Training Arguments

In [17]:
batch_size=32
training_args = TrainingArguments(
    output_dir="./results-focal-loss",
    num_train_epochs=6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_strategy="steps",
    logging_steps=200,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_macro_f1",
    greater_is_better=True,
    fp16=True,
    report_to="none",
    log_level="error",
    disable_tqdm=False,
    warmup_ratio=0.1,
    seed=42
)

## Initializing and runnning the trainer

In [18]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Macro F1,Weighted F1
1,0.000400,0.000493,0.086509,0.136208
2,0.000300,0.000495,0.125492,0.194192
3,0.000300,0.000448,0.150038,0.223082
4,0.000300,0.000471,0.141512,0.213162
5,0.000700,0.000428,0.115578,0.149367
6,0.000300,0.000486,0.158724,0.221342


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=7854, training_loss=0.0003853543489120345, metrics={'train_runtime': 7152.5099, 'train_samples_per_second': 70.263, 'train_steps_per_second': 1.098, 'total_flos': 3.329372744575488e+16, 'train_loss': 0.0003853543489120345, 'epoch': 6.0})

In [19]:
# import os
# from transformers import AutoModelForSequenceClassification
# training_args.num_train_epochs = 6
# last_check_point=os.listdir("./results-focal-loss")[-1]
# last_checkpoint = "./results-focal-loss/"+last_check_point
# model = AutoModelForSequenceClassification.from_pretrained(last_checkpoint)
# model.to(device)

# trainer = CustomTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_ds,
#     eval_dataset=eval_ds,
#     compute_metrics=compute_metrics,
# )
# trainer.train(resume_from_checkpoint=last_checkpoint)

In [20]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(
    model=model,          # the PyTorch model (DistilBERT) itself
    tokenizer=tokenizer,  # tokenizer used for training
    framework="pt",
    task="text-classification",
    device=0 if torch.cuda.is_available() else -1,  # int, not torch.device
    truncation=True,
    padding=True
)
predictions = []
ids=[]
test_ds=Dataset.from_pandas(test_df)
for item in test_ds:
    text = item["text"]
    # print(classifier(text))
    prediction = int(classifier(text)[0]["label"][-1])
    # print(prediction)

    # print(prediction)
    ids.append(item["id"])
    predictions.append(prediction)

In [21]:
submission=pd.DataFrame({
    "id":ids,
    "label":predictions
    })
submission.to_csv("submission.csv",index=False)